In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot


pyplot.style.use('seaborn')

In [ ]:
from numpy import (
    array,
    arange,
    concatenate,
    cos,
    cumsum,
    diff,
    linspace,
    pi,
    sin,
    where,
)


def plot_turn(linear_speed, angular_acceleration, max_angular_speed, turn_angle):
    time_period = 0.001
    angle = [0]
    tmpspeeds = [0]
    angular_speed = [0]
    x = [0]
    y = [0]
    elapsed_time = 0

    while True:
        if angle[-1] < turn_angle / 2.:
            new_angular_speed = min(angular_speed[-1] + angular_acceleration * time_period, max_angular_speed)
            tmpspeeds.append(new_angular_speed)
        else:
            if not len(tmpspeeds):
                break
            if len(tmpspeeds) == len(angular_speed):
                tmpspeeds.pop()
            new_angular_speed = tmpspeeds.pop()
        angular_speed.append(new_angular_speed)
        new_angle = angle[-1] + new_angular_speed * time_period
        angle.append(new_angle)
        x.append(x[-1] + linear_speed * cos(new_angle) * time_period)
        y.append(y[-1] + linear_speed * sin(new_angle) * time_period)
        elapsed_time += time_period

    omega = array(angular_speed)

    print('Raising time: %s' % where(diff(omega) < 0.)[0][0])
    print('Elapsed time: %s' % (len(omega) - 1))
    print('Final position: (%.5f, %.5f)' % (x[-1], y[-1]))

    pyplot.plot(omega)
    pyplot.show()

    pyplot.figure(figsize=(6, 6))
    pyplot.title('Final position: (%.5f, %.5f)' % (x[-1], y[-1]))
    pyplot.plot(x, y)
    pyplot.show()

## Search turn

In [ ]:
params = {
    'linear_speed': 0.4,
    'angular_acceleration': 32 * pi,
    'max_angular_speed': 2.35 * pi,
    'turn_angle': pi / 2,
}


plot_turn(**params)

## 90º turn

In [ ]:
params = {
    'linear_speed': .8,
    'angular_acceleration': 32 * pi,
    'max_angular_speed': 2.3 * pi,
    'turn_angle': pi / 2,
}


plot_turn(**params)

## 180º turn

In [ ]:
params = {
    'linear_speed': .7,
    'angular_acceleration': 32 * pi,
    'max_angular_speed': 2.5 * pi,
    'turn_angle': pi,
}


plot_turn(**params)

# Turn to 45º

In [ ]:
params = {
    'linear_speed': .71,
    'angular_acceleration': 32 * pi,
    'max_angular_speed': 2. * pi,
    'turn_angle': pi / 4.,
}


plot_turn(**params)

In [ ]:
before = 0.04978 - 0.11824
before

In [ ]:
after = (2 * (0.09 - 0.04978) ** 2) ** 0.5
after

## Turn to 135º

In [ ]:
params = {
    'linear_speed': .6,
    'angular_acceleration': 32 * pi,
    'max_angular_speed': 2.5 * pi,
    'turn_angle': 3 * pi / 4.,
}


plot_turn(**params)

In [ ]:
before = (0.18 - 0.14936) - 0.06075
before

In [ ]:
after = (2 * (0.18 - 0.14936) ** 2) ** 0.5
after

## Turn diagonal 90º

In [ ]:
params = {
    'linear_speed': .6,
    'angular_acceleration': 32 * pi,
    'max_angular_speed': 2.5 * pi,
    'turn_angle': pi / 2.,
}


plot_turn(**params)

Side:

In [ ]:
(2 * 0.09 ** 2) ** 0.5